In [2]:
#@title
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# coding=utf-8
import numpy as np
import pandas as pd

import codecs
import subprocess
from collections import namedtuple

import numpy as np
import pandas as pd
from gensim.models import Doc2Vec
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [4]:
############################ 加载数据 ############################
df_all = pd.read_csv("drive/My Drive/sougou/data/all_data.csv",encoding="UTF-8",index_col=0)
df_all['Age'] = df_all['Age'] - 1
df_all['Gender'] = df_all['Gender'] - 1
df_all['Education'] = df_all['Education'] - 1
df_all.head()

,ID,Age,Gender,Education,Query_List,Query_List1,jieba,jieba2
0,22DD920316420BE2DF8D6EE651BA174B,0.0,0.0,3.0,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,柔和双沟\t女生\t中财网首页 财经\t网址\t周公解梦大全查询2345\t曹云金再讽郭德纲...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再讽 ...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再...
1,43CC3AF5A8D6430A3B572337A889AFE4,1.0,0.0,2.0,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...","广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...",广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...,"广州 厨宝 烤箱 世情 薄 , 人情 恶 , 雨送 黄昏 花易落 , 晓 风干 , 泪痕 厦..."
2,6931EFC26D229CCFCEA125D3F3C21E57,3.0,1.0,2.0,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 壮观 衣服 网站 动漫 绘图 软件...,最 受欢迎 狗狗 排行榜 舶 怎么 读 场景 描 写 范例 三维 绘图 软件 枣 和 酸...
3,E780470C3BB0D340334BD08CDCC3C71A,1.0,1.0,3.0,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 怎么回事 麦当劳 旋风 勺子 吉林市 鹿...,干槽症 能 自愈 吗 太太 万岁 叶 舒心 去 没 去 美国 干槽症 右 眼皮 下面 一直 ...
4,0D58894CD481D4B3039000661665C0DB,2.0,1.0,2.0,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅 饲育 指南 韩姨 贵阳 古琴 洛基 风速 英美 店主 好吃 重生 儿童 寻家记 锦里 ...,貔貅 饲育 指南 韩姨 贵阳 学 古琴 洛基 风速 狗 综 英美 ] 店主 不 好吃 > 重...


In [7]:
df_all.shape

(188492, 8)

In [0]:
############################ 定义函数、类及变量 ############################
def run_cmd(cmd):
  print(cmd)
  process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
  for t, line in enumerate(iter(process.stdout.readline, b'')):
    line = line.decode('utf8').rstrip()
    print(line)
  process.communicate()
  return process.returncode


SentimentDocument = namedtuple('SentimentDocument', 'words tags')


class Doc_list(object):
  def __init__(self, f):
    self.f = f
  def __iter__(self):
    for i,line in enumerate(codecs.open(self.f,encoding='utf8')):
      words = line.strip().split(' ')
      tags = [int(words[0][2:])]
      words = words[1:]
      yield SentimentDocument(words,tags)

In [0]:
import time
def log(stri):
  now = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
  print(str(now) + ' ' + str(stri))

In [11]:
############################ 准备数据 ############################
train_num = 88492
test_num = 100000
doc_f = codecs.open("drive/My Drive/sougou/data/doc_for_d2v.txt", 'w', encoding='utf8')
for i, contents in enumerate(df_all.iloc[:(train_num+test_num)]['jieba2']):
  words = []
  for word in contents.split(' '):
    words.append(word)
  tags = [i]
  if i % 10000 == 0:
    log('iter = %d' % i)
  doc_f.write(u'_*{} {}\n'.format(i, ' '.join(words)))
doc_f.close()

2020-04-11 14:08:31 iter = 0
2020-04-11 14:08:32 iter = 10000
2020-04-11 14:08:33 iter = 20000
2020-04-11 14:08:35 iter = 30000
2020-04-11 14:08:36 iter = 40000
2020-04-11 14:08:37 iter = 50000
2020-04-11 14:08:38 iter = 60000
2020-04-11 14:08:39 iter = 70000
2020-04-11 14:08:41 iter = 80000
2020-04-11 14:08:42 iter = 90000
2020-04-11 14:08:43 iter = 100000
2020-04-11 14:08:44 iter = 110000
2020-04-11 14:08:45 iter = 120000
2020-04-11 14:08:47 iter = 130000
2020-04-11 14:08:48 iter = 140000
2020-04-11 14:08:49 iter = 150000
2020-04-11 14:08:50 iter = 160000
2020-04-11 14:08:51 iter = 170000
2020-04-11 14:08:53 iter = 180000


In [13]:
############################ dbow d2v ############################
d2v = Doc2Vec(dm=0, size=300, negative=5, hs=0, min_count=3, window=30, sample=1e-5, workers=8, alpha=0.025, min_alpha=0.025)
doc_list = Doc_list("drive/My Drive/sougou/data/doc_for_d2v.txt")
d2v.build_vocab(doc_list)

for i in range(2):
  log('pass: ' + str(i))
  #run_cmd('shuf alldata-id.txt > alldata-id-shuf.txt')
  doc_list = Doc_list("drive/My Drive/sougou/data/doc_for_d2v.txt")
  d2v.train(doc_list, total_examples=d2v.corpus_count, epochs=d2v.iter)
  X_d2v = np.array([d2v.docvecs[i] for i in range(train_num+test_num)])
  for k in ["Gender","Age","Education"]:
    scores = cross_val_score(LogisticRegression(C=3), X_d2v, df_all[k], cv=5)
    log("dbow of " + k + ":" + str(scores) + ' ' + str(np.mean(scores)))

d2v.save("drive/My Drive/sougou/model/dbow_d2v.model")
log('Save done!')

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


2020-04-11 14:35:11 pass: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://sc

2020-04-11 14:51:31 dbow of Gender:[0.50163134 0.51632669 0.52111518 0.5306117  0.5362884 ] 0.5211946630694677


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


...

2020-04-11 15:14:49 dbow of Age:[0.51057057 0.51821003 0.51766672 0.5248289  0.52069075] 0.5183933960390246


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


2020-04-11 15:17:18 dbow of Education:[0.51194992 0.51823656 0.51703008 0.52249456 0.52201708] 0.5183456406892828


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2020-04-11 15:17:25 Save done!


In [14]:
############################ dm d2v ############################
d2v = Doc2Vec(dm=1, size=300, negative=5, hs=0, min_count=3, window=30, sample=1e-5, workers=8, alpha=0.025, min_alpha=0.025)
doc_list = Doc_list("drive/My Drive/sougou/data/doc_for_d2v.txt")
d2v.build_vocab(doc_list)

for i in range(10):
  log('pass: ' + str(i))
  #run_cmd('shuf alldata-id.txt > alldata-id-shuf.txt')
  doc_list = Doc_list("drive/My Drive/sougou/data/doc_for_d2v.txt")
  d2v.train(doc_list, total_examples=d2v.corpus_count, epochs=d2v.iter)
  X_d2v = np.array([d2v.docvecs[i] for i in range(train_num+test_num)])
  for k in ["Gender","Age","Education"]:
    scores = cross_val_score(LogisticRegression(C=3), X_d2v, df_all[k], cv=5)
    log("dm of " + k + ":" + str(scores) + ' ' + str(np.mean(scores)))

d2v.save("drive/My Drive/sougou/model/dm_d2v.model")
log('Save done!')

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


2020-04-11 15:20:07 pass: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if __name__ == '__main__':


2020-04-11 15:46:10 dm of Gender:[0.50446961 0.52295817 0.53732293 0.53782694 0.53384795] 0.5272851216097234


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


...

2020-04-11 20:34:43 dm of Age:[0.51603491 0.51818351 0.51687092 0.51756061 0.51742798] 0.517215585875037


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


2020-04-11 20:37:17 dm of Education:[0.51802435 0.51876708 0.51724229 0.51909916 0.51718924] 0.5180644236223311


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2020-04-11 20:37:28 Save done!


In [4]:
# coding=utf-8
import numpy as np
import pandas as pd
from gensim.models import Doc2Vec
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
from sklearn.model_selection import StratifiedKFold

############################ 定义评估函数 ############################
def myAcc(y_true,y_pred):
  return np.mean(y_true == y_pred)

############################ 加载模型与数据模型 ############################
df_all = pd.read_csv("drive/My Drive/sougou/data/all_data.csv",encoding="UTF-8",index_col=0)
df_all['Age'] = df_all['Age'] - 1
df_all['Gender'] = df_all['Gender'] - 1
df_all['Education'] = df_all['Education'] - 1

train_num = 88492
test_num = 100000

model = Doc2Vec.load("drive/My Drive/sougou/model/dbow_d2v.model")
x_sp = np.array([model.docvecs[i] for i in range(train_num+test_num)])

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import time
def log(stri):
  now = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
  print(str(now) + ' ' + str(stri))

In [7]:
############################ dbowd2v stack ############################
############################ Gender ############################
np.random.seed(2020) # 固定种子，方便复现
df_stack = pd.DataFrame(index=range(len(df_all)))
tr_num = train_num
num_class = 2  
n = 5

x = x_sp[:tr_num]
y = df_all['Gender'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Gender'][tr_num:]

feat = 'dbowd2v'
stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    y_train = np_utils.to_categorical(y[tr], num_class)
#     y_test = np_utils.to_categorical(y_te, num_class)
    model = Sequential()
    model.add(Dense(300, input_shape=(x[tr].shape[1],)))
    model.add(Dropout(0.1))
    model.add(Activation('tanh'))
    model.add(Dense(num_class))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    history = model.fit(x[tr], y_train, shuffle=True,
                        batch_size=128, nb_epoch=35,
                        verbose=2)
    y_pred_va = model.predict_proba(x[va])
    y_pred_te = model.predict_proba(x_te)
    log('va acc:%f' % myAcc(y[va], model.predict_classes(x[va])))
#     log('te acc:%f' % myAcc(y_te, model.predict_classes(x_te)))
    score_va += myAcc(y[va], model.predict_classes(x[va]))
#     score_te += myAcc(y_te, model.predict_classes(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
for l in range(stack_all.shape[1]):
    df_stack['{}_{}_{}'.format(feat,"gender",l+1)] = stack_all[:, l]

df_stack.to_csv("drive/My Drive/sougou/data/dbowd2v_gender.csv", encoding='utf8', index=None)
log('Save dbowd2v stack done!')

2020-04-12 07:46:16 stack:1/5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/35
 - 2s - loss: 0.4195 - accuracy: 0.8377
Epoch 2/35
 - 2s - loss: 0.4065 - accuracy: 0.8448
Epoch 3/35
 - 2s - loss: 0.4021 - accuracy: 0.8468
Epoch 4/35
 - 2s - loss: 0.3985 - accuracy: 0.8477
Epoch 5/35
 - 2s - loss: 0.3954 - accuracy: 0.8482
Epoch 6/35
 - 2s - loss: 0.3921 - accuracy: 0.8496
Epoch 7/35
 - 2s - loss: 0.3895 - accuracy: 0.8495
Epoch 8/35
 - 2s - loss: 0.3875 - accuracy: 0.8506
Epoch 9/35
 - 2s - loss: 0.3850 - accuracy: 0.8507
Epoch 10/35
 - 2s - loss: 0.3836 - accuracy: 0.8518
Epoch 11/35
 - 2s - loss: 0.3817 - accuracy: 0.8524
Epoch 12/35
 - 2s - loss: 0.3807 - accuracy: 0.8530
Epoch 13/35
 - 2s - loss: 0.3789 - accuracy: 0.8529
Epoch 14/35
 - 2s - loss: 0.3777 - accuracy: 0.8542
Epoch 15/35
 - 2s - loss: 0.3756 - accuracy: 0.8546
Epoch 16/35
 - 2s - loss: 0.3744 - accuracy: 0.8555
Epoch 17/35
 - 2s - loss: 0.3723 - accuracy: 0.8555
Epoch 18/35
 - 2s - loss: 0.3714 - accuracy: 0.8562
Epoch 19/35
 - 2s - loss: 0.3696 - accuracy: 0.8561
Epoch 20/35
 - 2s - l

In [8]:
############################ dbowd2v stack ############################
############################ Age ############################
np.random.seed(2020) # 固定种子，方便复现
df_stack = pd.DataFrame(index=range(len(df_all)))
tr_num = train_num
num_class = 6  
n = 5

x = x_sp[:tr_num]
y = df_all['Age'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Age'][tr_num:]

feat = 'dbowd2v'
stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    y_train = np_utils.to_categorical(y[tr], num_class)
#     y_test = np_utils.to_categorical(y_te, num_class)
    model = Sequential()
    model.add(Dense(300, input_shape=(x[tr].shape[1],)))
    model.add(Dropout(0.1))
    model.add(Activation('tanh'))
    model.add(Dense(num_class))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    history = model.fit(x[tr], y_train, shuffle=True,
                        batch_size=128, nb_epoch=35,
                        verbose=2)
    y_pred_va = model.predict_proba(x[va])
    y_pred_te = model.predict_proba(x_te)
    log('va acc:%f' % myAcc(y[va], model.predict_classes(x[va])))
#     log('te acc:%f' % myAcc(y_te, model.predict_classes(x_te)))
    score_va += myAcc(y[va], model.predict_classes(x[va]))
#     score_te += myAcc(y_te, model.predict_classes(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
for l in range(stack_all.shape[1]):
    df_stack['{}_{}_{}'.format(feat,"age",l+1)] = stack_all[:, l]

df_stack.to_csv("drive/My Drive/sougou/data/dbowd2v_age.csv", encoding='utf8', index=None)
log('Save dbowd2v stack done!')

2020-04-12 07:52:01 stack:1/5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/35
 - 2s - loss: 1.0094 - accuracy: 0.5991
Epoch 2/35
 - 2s - loss: 0.9732 - accuracy: 0.6148
Epoch 3/35
 - 2s - loss: 0.9607 - accuracy: 0.6192
Epoch 4/35
 - 2s - loss: 0.9495 - accuracy: 0.6229
Epoch 5/35
 - 2s - loss: 0.9383 - accuracy: 0.6266
Epoch 6/35
 - 2s - loss: 0.9319 - accuracy: 0.6302
Epoch 7/35
 - 2s - loss: 0.9262 - accuracy: 0.6311
Epoch 8/35
 - 2s - loss: 0.9210 - accuracy: 0.6355
Epoch 9/35
 - 2s - loss: 0.9160 - accuracy: 0.6363
Epoch 10/35
 - 2s - loss: 0.9101 - accuracy: 0.6383
Epoch 11/35
 - 2s - loss: 0.9065 - accuracy: 0.6404
Epoch 12/35
 - 2s - loss: 0.9023 - accuracy: 0.6425
Epoch 13/35
 - 2s - loss: 0.8979 - accuracy: 0.6430
Epoch 14/35
 - 2s - loss: 0.8930 - accuracy: 0.6460
Epoch 15/35
 - 2s - loss: 0.8886 - accuracy: 0.6481
Epoch 16/35
 - 2s - loss: 0.8845 - accuracy: 0.6495
Epoch 17/35
 - 2s - loss: 0.8796 - accuracy: 0.6512
Epoch 18/35
 - 2s - loss: 0.8756 - accuracy: 0.6537
Epoch 19/35
 - 2s - loss: 0.8705 - accuracy: 0.6547
Epoch 20/35
 - 2s - l

In [9]:
############################ dbowd2v stack ############################
############################ Education ############################
np.random.seed(2020) # 固定种子，方便复现
df_stack = pd.DataFrame(index=range(len(df_all)))
tr_num = train_num
num_class = 6  
n = 5

x = x_sp[:tr_num]
y = df_all['Education'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Education'][tr_num:]

feat = 'dbowd2v'
stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    y_train = np_utils.to_categorical(y[tr], num_class)
#     y_test = np_utils.to_categorical(y_te, num_class)
    model = Sequential()
    model.add(Dense(300, input_shape=(x[tr].shape[1],)))
    model.add(Dropout(0.1))
    model.add(Activation('tanh'))
    model.add(Dense(num_class))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    history = model.fit(x[tr], y_train, shuffle=True,
                        batch_size=128, nb_epoch=35,
                        verbose=2)
    y_pred_va = model.predict_proba(x[va])
    y_pred_te = model.predict_proba(x_te)
    log('va acc:%f' % myAcc(y[va], model.predict_classes(x[va])))
#     log('te acc:%f' % myAcc(y_te, model.predict_classes(x_te)))
    score_va += myAcc(y[va], model.predict_classes(x[va]))
#     score_te += myAcc(y_te, model.predict_classes(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
for l in range(stack_all.shape[1]):
    df_stack['{}_{}_{}'.format(feat,"education",l+1)] = stack_all[:, l]

df_stack.to_csv("drive/My Drive/sougou/data/dbowd2v_education.csv", encoding='utf8', index=None)
log('Save dbowd2v stack done!')

2020-04-12 07:57:49 stack:1/5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/35
 - 2s - loss: 0.9524 - accuracy: 0.6156
Epoch 2/35
 - 2s - loss: 0.9199 - accuracy: 0.6301
Epoch 3/35
 - 2s - loss: 0.9102 - accuracy: 0.6339
Epoch 4/35
 - 2s - loss: 0.9028 - accuracy: 0.6371
Epoch 5/35
 - 2s - loss: 0.8962 - accuracy: 0.6382
Epoch 6/35
 - 2s - loss: 0.8901 - accuracy: 0.6428
Epoch 7/35
 - 2s - loss: 0.8857 - accuracy: 0.6433
Epoch 8/35
 - 2s - loss: 0.8795 - accuracy: 0.6458
Epoch 9/35
 - 2s - loss: 0.8764 - accuracy: 0.6466
Epoch 10/35
 - 2s - loss: 0.8723 - accuracy: 0.6495
Epoch 11/35
 - 2s - loss: 0.8675 - accuracy: 0.6513
Epoch 12/35
 - 2s - loss: 0.8623 - accuracy: 0.6531
Epoch 13/35
 - 2s - loss: 0.8593 - accuracy: 0.6561
Epoch 14/35
 - 2s - loss: 0.8551 - accuracy: 0.6556
Epoch 15/35
 - 2s - loss: 0.8504 - accuracy: 0.6593
Epoch 16/35
 - 2s - loss: 0.8468 - accuracy: 0.6601
Epoch 17/35
 - 2s - loss: 0.8431 - accuracy: 0.6615
Epoch 18/35
 - 2s - loss: 0.8390 - accuracy: 0.6639
Epoch 19/35
 - 2s - loss: 0.8338 - accuracy: 0.6647
Epoch 20/35
 - 2s - l

In [10]:
############################ 加载模型 ############################
model = Doc2Vec.load("drive/My Drive/sougou/model/dm_d2v.model")
x_sp = np.array([model.docvecs[i] for i in range(train_num+test_num)])

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
############################ dmd2v stack ############################
############################ Gender ############################
np.random.seed(2020) # 固定种子，方便复现
df_stack = pd.DataFrame(index=range(len(df_all)))
tr_num = train_num
num_class = 2  
n = 5

x = x_sp[:tr_num]
y = df_all['Gender'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Gender'][tr_num:]

feat = 'dmd2v'
stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    y_train = np_utils.to_categorical(y[tr], num_class)
#     y_test = np_utils.to_categorical(y_te, num_class)
    model = Sequential()
    model.add(Dense(300, input_shape=(x[tr].shape[1],)))
    model.add(Dropout(0.1))
    model.add(Activation('tanh'))
    model.add(Dense(num_class))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    history = model.fit(x[tr], y_train, shuffle=True,
                        batch_size=128, nb_epoch=35,
                        verbose=2)
    y_pred_va = model.predict_proba(x[va])
    y_pred_te = model.predict_proba(x_te)
    log('va acc:%f' % myAcc(y[va], model.predict_classes(x[va])))
#     log('te acc:%f' % myAcc(y_te, model.predict_classes(x_te)))
    score_va += myAcc(y[va], model.predict_classes(x[va]))
#     score_te += myAcc(y_te, model.predict_classes(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
for l in range(stack_all.shape[1]):
    df_stack['{}_{}_{}'.format(feat,"gender",l+1)] = stack_all[:, l]

df_stack.to_csv("drive/My Drive/sougou/data/dmd2v_gender.csv", encoding='utf8', index=None)
log('Save dmd2v stack done!')

2020-04-12 08:11:48 stack:1/5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/35
 - 2s - loss: 0.4944 - accuracy: 0.7831
Epoch 2/35
 - 2s - loss: 0.4403 - accuracy: 0.8122
Epoch 3/35
 - 2s - loss: 0.4157 - accuracy: 0.8245
Epoch 4/35
 - 2s - loss: 0.3855 - accuracy: 0.8393
Epoch 5/35
 - 2s - loss: 0.3464 - accuracy: 0.8573
Epoch 6/35
 - 2s - loss: 0.3080 - accuracy: 0.8734
Epoch 7/35
 - 2s - loss: 0.2720 - accuracy: 0.8887
Epoch 8/35
 - 2s - loss: 0.2407 - accuracy: 0.9017
Epoch 9/35
 - 2s - loss: 0.2160 - accuracy: 0.9126
Epoch 10/35
 - 2s - loss: 0.1893 - accuracy: 0.9248
Epoch 11/35
 - 2s - loss: 0.1700 - accuracy: 0.9333
Epoch 12/35
 - 2s - loss: 0.1543 - accuracy: 0.9399
Epoch 13/35
 - 2s - loss: 0.1441 - accuracy: 0.9431
Epoch 14/35
 - 2s - loss: 0.1268 - accuracy: 0.9504
Epoch 15/35
 - 2s - loss: 0.1223 - accuracy: 0.9523
Epoch 16/35
 - 2s - loss: 0.1102 - accuracy: 0.9580
Epoch 17/35
 - 2s - loss: 0.1020 - accuracy: 0.9610
Epoch 18/35
 - 2s - loss: 0.1008 - accuracy: 0.9605
Epoch 19/35
 - 2s - loss: 0.0926 - accuracy: 0.9644
Epoch 20/35
 - 2s - l

In [12]:
############################ dmd2v stack ############################
############################ Age ############################
np.random.seed(2020) # 固定种子，方便复现
df_stack = pd.DataFrame(index=range(len(df_all)))
tr_num = train_num
num_class = 6  
n = 5

x = x_sp[:tr_num]
y = df_all['Age'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Age'][tr_num:]

feat = 'dmd2v'
stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    y_train = np_utils.to_categorical(y[tr], num_class)
#     y_test = np_utils.to_categorical(y_te, num_class)
    model = Sequential()
    model.add(Dense(300, input_shape=(x[tr].shape[1],)))
    model.add(Dropout(0.1))
    model.add(Activation('tanh'))
    model.add(Dense(num_class))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    history = model.fit(x[tr], y_train, shuffle=True,
                        batch_size=128, nb_epoch=35,
                        verbose=2)
    y_pred_va = model.predict_proba(x[va])
    y_pred_te = model.predict_proba(x_te)
    log('va acc:%f' % myAcc(y[va], model.predict_classes(x[va])))
#     log('te acc:%f' % myAcc(y_te, model.predict_classes(x_te)))
    score_va += myAcc(y[va], model.predict_classes(x[va]))
#     score_te += myAcc(y_te, model.predict_classes(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
for l in range(stack_all.shape[1]):
    df_stack['{}_{}_{}'.format(feat,"age",l+1)] = stack_all[:, l]

df_stack.to_csv("drive/My Drive/sougou/data/dmd2v_age.csv", encoding='utf8', index=None)
log('Save dmd2v stack done!')

2020-04-12 08:17:50 stack:1/5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/35
 - 2s - loss: 1.1180 - accuracy: 0.5507
Epoch 2/35
 - 2s - loss: 0.9997 - accuracy: 0.5983
Epoch 3/35
 - 2s - loss: 0.9498 - accuracy: 0.6230
Epoch 4/35
 - 2s - loss: 0.9027 - accuracy: 0.6431
Epoch 5/35
 - 2s - loss: 0.8518 - accuracy: 0.6676
Epoch 6/35
 - 2s - loss: 0.8037 - accuracy: 0.6898
Epoch 7/35
 - 2s - loss: 0.7600 - accuracy: 0.7092
Epoch 8/35
 - 2s - loss: 0.7107 - accuracy: 0.7341
Epoch 9/35
 - 2s - loss: 0.6780 - accuracy: 0.7443
Epoch 10/35
 - 2s - loss: 0.6404 - accuracy: 0.7570
Epoch 11/35
 - 2s - loss: 0.6132 - accuracy: 0.7703
Epoch 12/35
 - 2s - loss: 0.5885 - accuracy: 0.7791
Epoch 13/35
 - 2s - loss: 0.5580 - accuracy: 0.7926
Epoch 14/35
 - 2s - loss: 0.5397 - accuracy: 0.7968
Epoch 15/35
 - 2s - loss: 0.5197 - accuracy: 0.8071
Epoch 16/35
 - 2s - loss: 0.5048 - accuracy: 0.8116
Epoch 17/35
 - 2s - loss: 0.4859 - accuracy: 0.8188
Epoch 18/35
 - 2s - loss: 0.4717 - accuracy: 0.8226
Epoch 19/35
 - 2s - loss: 0.4585 - accuracy: 0.8275
Epoch 20/35
 - 2s - l

In [13]:
############################ dmd2v stack ############################
############################ Education ############################
np.random.seed(2020) # 固定种子，方便复现
df_stack = pd.DataFrame(index=range(len(df_all)))
tr_num = train_num
num_class = 6  
n = 5

x = x_sp[:tr_num]
y = df_all['Education'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Education'][tr_num:]

feat = 'dmd2v'
stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    y_train = np_utils.to_categorical(y[tr], num_class)
#     y_test = np_utils.to_categorical(y_te, num_class)
    model = Sequential()
    model.add(Dense(300, input_shape=(x[tr].shape[1],)))
    model.add(Dropout(0.1))
    model.add(Activation('tanh'))
    model.add(Dense(num_class))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    history = model.fit(x[tr], y_train, shuffle=True,
                        batch_size=128, nb_epoch=35,
                        verbose=2)
    y_pred_va = model.predict_proba(x[va])
    y_pred_te = model.predict_proba(x_te)
    log('va acc:%f' % myAcc(y[va], model.predict_classes(x[va])))
#     log('te acc:%f' % myAcc(y_te, model.predict_classes(x_te)))
    score_va += myAcc(y[va], model.predict_classes(x[va]))
#     score_te += myAcc(y_te, model.predict_classes(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
for l in range(stack_all.shape[1]):
    df_stack['{}_{}_{}'.format(feat,"education",l+1)] = stack_all[:, l]

df_stack.to_csv("drive/My Drive/sougou/data/dmd2v_education.csv", encoding='utf8', index=None)
log('Save dmd2v stack done!')

2020-04-12 08:23:53 stack:1/5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/35
 - 2s - loss: 1.0518 - accuracy: 0.5698
Epoch 2/35
 - 2s - loss: 0.9422 - accuracy: 0.6140
Epoch 3/35
 - 2s - loss: 0.8959 - accuracy: 0.6354
Epoch 4/35
 - 2s - loss: 0.8506 - accuracy: 0.6591
Epoch 5/35
 - 2s - loss: 0.8030 - accuracy: 0.6802
Epoch 6/35
 - 2s - loss: 0.7541 - accuracy: 0.7028
Epoch 7/35
 - 2s - loss: 0.7071 - accuracy: 0.7238
Epoch 8/35
 - 2s - loss: 0.6670 - accuracy: 0.7409
Epoch 9/35
 - 2s - loss: 0.6294 - accuracy: 0.7582
Epoch 10/35
 - 2s - loss: 0.5982 - accuracy: 0.7716
Epoch 11/35
 - 2s - loss: 0.5685 - accuracy: 0.7815
Epoch 12/35
 - 2s - loss: 0.5393 - accuracy: 0.7941
Epoch 13/35
 - 2s - loss: 0.5135 - accuracy: 0.8035
Epoch 14/35
 - 2s - loss: 0.4915 - accuracy: 0.8119
Epoch 15/35
 - 2s - loss: 0.4747 - accuracy: 0.8188
Epoch 16/35
 - 2s - loss: 0.4578 - accuracy: 0.8259
Epoch 17/35
 - 2s - loss: 0.4405 - accuracy: 0.8317
Epoch 18/35
 - 2s - loss: 0.4235 - accuracy: 0.8400
Epoch 19/35
 - 2s - loss: 0.4082 - accuracy: 0.8458
Epoch 20/35
 - 2s - l